In [9]:
import os
import pandas as pd

base_path='aclImdb'
processed_data_path='processed_data'

train_data_path = base_path + '/train'
test_data_path = base_path + '/test'

def read_data(path,sentiment):
    dic={'text':[],'label':[]}
    for file_path in os.listdir(path):
        with open(os.path.join(path,file_path),encoding='utf8') as f:
            text=f.readline()
        dic['text'].append(text)
        dic['label'].append(sentiment)
    df=pd.DataFrame(dic)
    return df

def get_data(raw_data_path,saved_path):
    if os.path.exists(saved_path):
        df=pd.read_csv(saved_path)
    else:
        df_pos=read_data(raw_data_path+'/pos',1)
        df_neg=read_data(raw_data_path+'/neg',0)
        df = pd.concat([df_pos,df_neg],ignore_index=True)
        df.to_csv(saved_path,index=False)
    return df

In [10]:
df_train = get_data(train_data_path,processed_data_path + '/train.csv')

In [11]:
df_test = get_data(test_data_path,processed_data_path+'/test.csv')

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
logistic_regressor = LogisticRegression()
tf_idf_vectorizer = TfidfVectorizer()
pipeline = Pipeline([
    ('vectorizer',tf_idf_vectorizer),
    ('model',logistic_regressor)
])
pipeline.fit(df_train.iloc[[0,-1,-2,1]]['text'],df_train.iloc[[0,-1,-2,1]]['label'])

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('model', LogisticRegression())])

In [83]:
pred_train_label = pipeline.predict(df_train['text'])
from sklearn.metrics import accuracy
print('Training accuracy',accuracy(pred_train_label,df_train['label']))

array([0, 0, 0, 0], dtype=int64)

In [79]:
pred_test_label = pipeline.predict(df_test['text'])
print('Testing accuracy',accuracy(pred_test_label,df_test['label']))

,text,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
24999,"This is one of the dumbest films, I've ever se...",0
24998,Some films that you pick up for a pound turn o...,0
1,Homelessness (or Houselessness as George Carli...,1
